In [ ]:
import csv
import json
import os
import re
import time

import gym
import roboschool

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from gym import wrappers
from ipywidgets import Video
import ipywidgets as widgets
from multiprocessing import Pool, Process
from IPython.display import display

In [ ]:
# %load load-model.py
def load_model(model_path):   
    import tensorflow as tf

    class Normc_initializer(tf.keras.initializers.Initializer):
        def __init__(self, std=1.0):
            self.std=std

        def __call__(self, shape, dtype=None, partition_info=None):
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
    
    class ObservationNormalizationLayer(tf.keras.layers.Layer):
        def __init__(self, ob_mean, ob_std, **kwargs):
            self.ob_mean = ob_mean
            self.ob_std = ob_std
            super(ObservationNormalizationLayer, self).__init__(**kwargs)

        def call(self, x):
            return tf.clip_by_value((x - self.ob_mean) / self.ob_std, -5.0, 5.0)
        
        # get_config and from_config need to implemented to be able to serialize the model
        def get_config(self):
            base_config = super(ObservationNormalizationLayer, self).get_config()
            base_config['ob_mean'] = self.ob_mean
            base_config['ob_std'] = self.ob_std
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
        
    class DiscretizeActionsUniformLayer(tf.keras.layers.Layer):
        def __init__(self, num_ac_bins, adim, ahigh, alow, **kwargs):
            self.num_ac_bins = num_ac_bins
            self.adim = adim
            # ahigh, alow are NumPy arrays when extracting from the environment, but when the model is loaded from a h5
            # File they get initialised as a normal list, where operations like subtraction does not work, thereforce
            # cast them explicitly
            self.ahigh = np.array(ahigh)
            self.alow = np.array(alow)
            super(DiscretizeActionsUniformLayer, self).__init__(**kwargs)

        def call(self, x):            
            # Reshape to [n x i x j] where n is dynamically chosen, i equals action dimension and j equals the number
            # of bins
            scores_nab = tf.reshape(x, [-1, self.adim, self.num_ac_bins])
            # This picks the bin with the greatest value
            a = tf.argmax(scores_nab, 2)
            
            # Then transform the interval from [0, num_ac_bins - 1] to [-1, 1] which equals alow and ahigh
            ac_range_1a = (self.ahigh - self.alow)[None, :]
            return 1. / (self.num_ac_bins - 1.) * tf.keras.backend.cast(a, 'float32') * ac_range_1a + self.alow[None, :]        
        
        # get_config and from_config need to implemented to be able to serialize the model
        def get_config(self):
            base_config = super(DiscretizeActionsUniformLayer, self).get_config()
            base_config['num_ac_bins'] = self.num_ac_bins
            base_config['adim'] = self.adim
            base_config['ahigh'] = self.ahigh
            base_config['alow'] = self.alow
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
    
    custom_objects = {'Normc_initializer' : Normc_initializer, 
                      'ObservationNormalizationLayer' : ObservationNormalizationLayer,
                      'DiscretizeActionsUniformLayer' : DiscretizeActionsUniformLayer}
    
    try:
        model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
    except OSError as e:
        print(e)
        return None
    return model

In [ ]:
def rollout_evaluation(env, model, render=False, timestep_limit=None, random_stream=None):
    """
    If random_stream is provided, the rollout will take noisy actions with noise drawn from that stream.
    Otherwise, no action noise will be added.
    """

    env_timestep_limit = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    timestep_limit = env_timestep_limit if timestep_limit is None else min(timestep_limit, env_timestep_limit)
    rews = []
    t = 0
    ob = env.reset()
    obs = []
    predictions=[]
    for _ in range(timestep_limit):
        if render:
            env.render()
        obs.append(ob[None])
        pred = model.predict_on_batch(ob[None])
        predictions.append(pred)
        ac = pred[0]
        try:
            ob, rew, done, _ = env.step(ac)
        except AssertionError:
            # Is thrown when for example ac is a list which has at least one entry with NaN
            raise 
        rews.append(rew)
        t += 1

        if done:
            break
    x_test = np.concatenate(obs)
    y_test = np.concatenate(predictions)
    np.savez_compressed('x_test', x_test)
    np.savez_compressed('y_test', y_test)
    return np.array(rews, dtype=np.float32), t


def run_model(model_file_path, model_file, save_directory, record=False):   
    
        with open(os.path.join(model_file_path, "config.json"), encoding='utf-8') as f:
            config = json.load(f)
    
        env = gym.make(config['config']['env_id'])
        env.reset()
        if record:
            env = wrappers.Monitor(env, save_directory, force=True)

        model = load_model(os.path.join(model_file_path, model_file))
        
        try:
            rewards, length = rollout_evaluation(env, model)
        except AssertionError:
            print("The model file provided produces non finite numbers. Stopping.")
            return
        
        env.close()
        print(rewards)
        print([rewards.sum(), length])

        return [rewards.sum(), length]

In [ ]:
model_file_path = "/home/jovyan/base_repository/Workspace/ann_training_run/"
model_file_name = "snapshot_00423.h5"

# Lets store the video file in the same directory as the model file
save_directory = model_file_path


#with Pool(os.cpu_count()) as pool:
#    pool.apply(func=run_model, args=(model_file_path, model_file_name, save_directory, True))

run_model(model_file_path, model_file_name, save_directory, False)


In [ ]:
#!sleep 2
for file in os.listdir(save_directory):
    if file.endswith('.mp4'):
        video_file = os.path.join(save_directory, file)
        print(file)
video = Video.from_file(video_file)
display(video)